# Introduction to scRNA-seq
- Much of this notebook is adapted from the Seurat vignettes https://satijalab.org/seurat and GitHub repository https://github.com/satijalab/seurat

### How does scRNA-seq differ from bulk RNA-seq?

<img src="src/bulk_vs_singlecell.png" width="500">

- In bulk RNA-seq you are taking a snapshot of expression of all the cells in a sample and your measurements are aggregated across all of those cells.
- In scRNA-seq, you can get a sense of the heterogeneity of the cells in your sample.
    - Are there novel or rare cell types?
    - What about cell type specific gene expression?
    - Does the distribution of different cell types change across time or treatment?
- This increased resolution comes with some unique challenges.
    - Dropouts - genes that are not detected in some cells, can lead to sparse expression matrices with many zero values.
    - Doublets - sequencing two cells at the same time and can't distinguish their expression or cell types, need to filter these out during QC.
    - Dying cells - you will lose some cells because they are dead or dying, you can also filter these out during sample QC.
    - You also should be cautious when thinking about your sample sizes. For example, you may be sequencing thousands of cells but if they all come from the same mouse you lose the ability to generalize your findings.
   
### scRNA-seq technologies
- Although 10X genomics is probably the most popular technology for scRNA-seq, there are other flavors (see PMID 30472192 and PMID 28212749).   

<img src="src/10x_flow.png" width="500">

- 10x sequencing encapsulates a cell, reagents, and a bead w/ primer in an oil droplet (aka GEM or Gel Bead-in EMulsion).    

<img src="src/10x_bead.png" width="500">

- After encapsulation of cells, beads, and reagents in the oil droplets, the bead is dissolved and releases primers. 
- The poly (dT) primers are used for generating the gene expression libraries. 
- The capture sequence primers are shown in a lighter shading because they are only used in situations where you'd like to add an extra channel of information to your experiment by using feature barcoding (cell-surface protein characterization, multiplexing, etc).
- https://teichlab.github.io/scg_lib_structs/ is an excellent resource for information about the resulting library structures for 10x libraries (and other single cell technologies like Drop-seq or SMART-seq).

### Seurat objects
- This workshop focuses on using Seurat objects to structure your scRNA-seq data (https://github.com/satijalab/seurat/wiki/Seurat).     

<img src="src/seurat_object.png" width="500">

- Each Seurat object is composed of different slots.
    - **`assays`** is a list of all the assays in the object.
        - Typically only has the `RNA` assay, but you can add others (like `SCT` shown in the figure above, could also be antibody-derived tags, etc.). 
        - You can see which assay is the currently active assay by looking in the `active.assay` slot and switch between them using the `DefaultAssay()` function.
        - Each assay will store multiple transformations of the data in different slots -- in the case of `RNA` data these slots are: 
            - `@counts` contains the raw counts.
            - `@data` contains the normalized counts.
            - `@scale.data` contains the scaled data for dimensional reduction.
        - The slots store the data as a sparse matrix where the rows are gene and the columns are cells.      
    - **`meta.data`** is a matrix of all the cell-level metadata.
        - This will include information about which condition, timepoint, batch, etc. a for a given cell.
        - It also includes metrics that will be relevant for QC, like `nCount_RNA` and `nFeature_RNA`
            - `nCount_RNA` is the total number of molecules (UMIs) detected within a cell.
            - `nFeature_RNA` is the total number of genes detected within a cell.
        - Once you have completed clustering, you'll also see information about which cluster each cell has been assigned to.
        - The different categories or columns in the `meta.data` are also called `Idents` in Seurat.
        - You can see the current `Ident` in the `active.ident` slot and switch between them using the `Idents()` function (this will probably be important for running differential expression testing).
    - **`graphs`** is a list of the nearest neighbor graphs.
        - The objects stored in `graphs` are cell x cell matrices containing the neighborhood overlap (Jaccard index) between every cell and its nearest neighbors. 
    - **`reductions`** is a list of `DimReduc` objects.
    - **`version`** contains information about which version of Seurat was used to make the object.
    - There are other optional slots, including **`tools`** and **`misc`** that can be populated by specific analysis tools (`tools`) or users can store their own additional information (`misc`).

### Importing Data

Set a seed and import packages, set libPaths 

In [6]:
set.seed(61)
.libPaths(c('/usr/local/lib/R/site-library', '/usr/local/lib/R/library'))
library(RColorBrewer)
library(Seurat)
library(patchwork)
library(ggplot2)
library(dplyr)
library(hdf5r)
library(stringr)
library(biomaRt)
library(kableExtra)
library(SeuratDisk)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in library(hdf5r): there is no package called ‘hdf5r’


We will use some of the PBMC data that 10x makes available to the public, they were downloaded using wget:

In [ ]:
#wget https://cf.10xgenomics.com/samples/cell-exp/3.0.2/5k_pbmc_v3/5k_pbmc_v3_filtered_feature_bc_matrix.h5
#wget https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_filtered_feature_bc_matrix.h5

We use `Read10X_h5` to import the data, you could also use `Read10X` and give a path to a folder that contains your matrix, features, and barcode tsv files.

In [ ]:
#data_dir <- '/gpfs/data/cbc/scrna_r_workshop'
pbmc.1k <- Read10X_h5(paste0(data_dir, '/data/pbmc_1k_v3_filtered_feature_bc_matrix.h5'))
pbmc.5k <- Read10X_h5(paste0(data_dir, '/data/5k_pbmc_v3_filtered_feature_bc_matrix.h5'))

Then we create the Seurat objects:

In [ ]:
pbmc.1k <- CreateSeuratObject(counts = pbmc.1k, project = 'pbmc.1k')
pbmc.5k <- CreateSeuratObject(counts = pbmc.5k, project = 'pbmc.5k')

We will `merge()` the objects for now. This will create a new Seurat object that simply concatenates the counts from the two objects.

In [ ]:
all_data <- merge(x = pbmc.1k, y = c(pbmc.5k), project = 'pbmc')

### Data QC and filtering

Our data might have captured dead or dying cells. We can use the percentage of mitochondrial reads from a cell as a proxy for the health/quality of a cell. Dead or dying cells will have a relatively higher proportion of mitochondrial reads. We can use the `[[]]` notation and the `PercentageFeatureSet` function to add this information to the `meta.data` slot.

In [ ]:
all_data[["percent.mt"]] <- PercentageFeatureSet(all_data, pattern = "^MT-")

Note that the format denoting the mitochondrial sequences might vary depending on the genome or organism used (it might be `mt-`, for example

We can set the order of the idents before we plot to make sure we are plotting things in the right order

In [ ]:
Idents(all_data) <- 'orig.ident'
levels(all_data) <- c("pbmc.1k", "pbmc.5k")

Use `VlnPlot`:

In [ ]:
VlnPlot(all_data, features = "nFeature_RNA")
VlnPlot(all_data, features = "nCount_RNA")
VlnPlot(all_data, features="percent.mt")

Use `FeatureScatter`:

In [ ]:
FeatureScatter(all_data, feature1 = "nCount_RNA", feature2 = "percent.mt")
FeatureScatter(all_data, feature1 = "nFeature_RNA", feature2 = "percent.mt")
FeatureScatter(all_data, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

Pick your filtering cutoffs based on the results of the plots:

In [ ]:
all_data_sub <- subset(all_data, subset = nFeature_RNA > 500 & nFeature_RNA < 6000 & percent.mt < 25)

We suggest using SCTransform to normalize your data. This is an alternative to the `NormalizeData`, `FindVariableFeatures`, and `ScaleData` workflow.

First, split the merged obejct by `orig.ident`:

In [ ]:
all_data_split <- SplitObject(all_data_sub, split.by = 'orig.ident')

### SCTransform normalization, clustering, dimension reduction

#### Theory

The theory behind sctransform is very similar to the generalized linear models (GLMs) used in bulkRNAseq analysis packages like `DESeq2` and `edgeR`. In `DESeq2` a negative binomial model is fitted to the counts and the mean and dispersion (roughly speaking how variable the observed count will be from the mean count) estimates from that model are used as the test statistics for comparison between groups. The same idea applies with `sctransform`, with an additional aspect where `sctransform` pools information across genes with similar abundances in order to address the higher sparsity of single cell data.

Below is a side-by-side comparison of `sctransform` with `NormalizeData`, `FindVariableFeatures` and `ScaleData` on the PBMC3k data:

<img src="src/sctransform_vs_regular.png" width="800">

Then call the SCTransform function on the list you just made:

In [ ]:
all_data_list <- lapply(all_data_split, function(x) {
    x <- SCTransform(x,verbose=FALSE)}) # if you run this you might get several iteration limit reached issues

Then we can re-merge the data using `merge.data = TRUE` so that we also merge the data slots instead of just merging the counts (which requires renormalization). This is recommended if the same normalization approach was applied to all objects.

In [ ]:
all_data_merged <- merge(x = all_data_list$pbmc.1k, y = c(all_data_list$pbmc.5k), merge.data = TRUE)

Then run PCA

In [ ]:
all_data_merged <- RunPCA(all_data_merged)

You are getting an error because after the merge, the variable feature slot gets wiped since there could be different variable features in each original object. The default for `RunPCA` is to use those features and since it's empty, you get an error. You can either set the variable features of the merged SCT assay yourself (to something like the intersection or union of the individual object's variable features) or provide this vector of features to RunPCA itself.
#https://github.com/satijalab/seurat/issues/2852
We will run `SelectIntegrationFeatures` from the list of Seurat objects before merge and then assign those as the `VariableFeatures`

In [ ]:
integration_features <- SelectIntegrationFeatures(all_data_list)
VariableFeatures(all_data_merged) <- integration_features

Now we can run the PCA and make an elbow plot:

In [ ]:
all_data_merged <- RunPCA(all_data_merged)
ElbowPlot(all_data_merged)

Based on this plot, we get diminishing information returned once we get above ~10 PCs.

Next, construct a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the FindNeighbors() function, and takes as input the previously defined dimensionality of the dataset (first 10 PCs).

In [ ]:
all_data_merged <- FindNeighbors(all_data_merged, dims = 1:10)

To cluster the cells, weapply the Louvain algorithm to iteratively group cells together, with the goal of optimizing the standard modularity function. The FindClusters() function implements this procedure, and contains a resolution parameter that sets the ‘granularity’ of the downstream clustering, with increased values leading to a greater number of clusters. Setting this parameter between 0.4-1.2 typically returns good results for single-cell datasets of around 3K cells. Optimal resolution often increases for larger datasets. The clusters can be found using the Idents() function.

In [ ]:
all_data_merged <- FindClusters(all_data_merged)

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, we suggest using the same PCs as input to the clustering analysis. We recommend using FIt-SNE: Use the FFT-accelerated Interpolation-based t-SNE. Based on Kluger Lab code found here: https://github.com/KlugerLab/FIt-SNE to make things faster.

In [ ]:
all_data_merged <- RunUMAP(all_data_merged, dims = 1:10)
all_data_merged <- RunTSNE(all_data_merged, tsne.method = "FIt-SNE", seed.use=61)

We can look at the clusters using `DimPlot()` function:

In [ ]:
DimPlot(all_data_merged, reduction='tsne')
DimPlot(all_data_merged, reduction='umap')

We can also group the plots by the `idents`

In [ ]:
DimPlot(all_data_merged, reduction='tsne', group.by = 'orig.ident')
DimPlot(all_data_merged, reduction='umap', group.by = 'orig.ident')

### Data Integration

In most cases if we are looking for differences in gene expression between experimental conditions, we will actually want to integrate our datasets rather than merge them. This is because the experimental conditions can cause cells to cluster both by condition and by cell type. Merging should really only be used for technical replicates with a low batch effect.

There are two choices with integrating data: RPCA and CCA.

"Instead of utilizing canonical correlation analysis (‘CCA’) to identify anchors, we instead utilize reciprocal PCA (‘RPCA’). When determining anchors between any two datasets using RPCA, we project each dataset into the others PCA space and constrain the anchors by the same mutual neighborhood requirement. The commands for both workflows are largely identical, but the two methods may be applied in different context.

By identifying shared sources of variation between datasets, CCA is well-suited for identifying anchors when cell types are conserved, but there are very substantial differences in gene expression across experiments. CCA-based integration therefore enables integrative analysis when experimental conditions or disease states introduce very strong expression shifts, or when integrating datasets across modalities and species. However, CCA-based integration may also lead to overcorrection, especially when a large proportion of cells are non-overlapping across datasets.

RPCA-based integration runs significantly faster, and also represents a more conservative approach where cells in different biological states are less likely to ‘align’ after integration. We therefore,recommend RPCA during integrative analysis where: * A substantial fraction of cells in one dataset have no matching type in the other * Datasets originate from the same platform (i.e. multiple lanes of 10x genomics) * There are a large number of datasets or cells to integrate (see INSERT LINK for more tips on integrating large datasets)

Below, we demonstrate the use of reciprocal PCA to align the same stimulated and resting datasets first analyzed in our introduction to scRNA-seq integration vignette. While the list of commands is nearly identical, this workflow requires users to run principal components analysis (PCA) individually on each dataset prior to integration. Users should also set the ‘reduction’ argument to ‘rpca’, when running FindIntegrationAnchors()."


First, run `PrepSCTIntegration`:

In [ ]:
all_data_list <- PrepSCTIntegration(object.list = all_data_list, anchor.features = integration_features)

Run PCA on all objects:

In [ ]:
all_data_list <- lapply(X = all_data_list, FUN = RunPCA, features = integration_features)

Find a set of anchors between seurat objects in a list. These anchors can later be used to integrate the objects using the IntegrateData function. 

In [ ]:
anchors <- FindIntegrationAnchors(object.list = all_data_list, normalization.method = "SCT", anchor.features = integration_features, reduction = 'rpca')

Integrate data

In [ ]:
all_data_integrated <- IntegrateData(anchorset = anchors, normalization.method = "SCT")

Run PCA, clustering, and dimension reduction on the new integrated object:

In [ ]:
all_data_integrated <- RunPCA(all_data_integrated)
all_data_integrated <- FindNeighbors(all_data_integrated, dims = 1:10)
all_data_integrated <- FindClusters(all_data_integrated)
all_data_integrated <- RunUMAP(all_data_integrated, dims = 1:10)
all_data_integrated <- RunTSNE(all_data_integrated, tsne.method = "FIt-SNE", seed.use=61)

Compare merged vs integrated:

In [ ]:
DimPlot(all_data_merged, reduction='tsne', group.by = 'orig.ident') + ggtitle("Merged") | DimPlot(all_data_integrated, reduction='tsne', group.by = 'orig.ident') + ggtitle("Integrated")
DimPlot(all_data_merged, reduction='umap', group.by = 'orig.ident') + ggtitle("Merged") | DimPlot(all_data_integrated, reduction='umap', group.by = 'orig.ident') + ggtitle("Integrated")

### Differential Expression Testing

Seurat has a few methods for finding differentially expressed genes across cells:

`FindMarkers`: Find markers (differentially expressed genes) for two specific identity classes.
`FindAllMarkers`: Find markers for each identity class by comparing each class to all of the others.
`FindConservedMarkers` Find markers conserved between two groups.

Lets look at differentially expressed genes across Seurat clusters. First, change the default identity class:

In [ ]:
Idents(all_data_integrated) <- 'seurat_clusters'

Make sure we are using the `RNA` assay:

In [ ]:
DefaultAssay(all_data_integrated) <- 'RNA'

Run `FindAllMarkers` with some pre-filtering to look at features that have at least a two-fold change in average expression in each comparison and features that are detected in at least 95% of cells in either group.

In [ ]:
cluster_markers <- FindAllMarkers(all_data_integrated, min.pct = 0.95, logfc.threshold = log(2))

The object created has the following columns of information:
`avg_logFC`: log fold-chage of the average expression between the two groups. Positive values indicate that the gene is more highly expressed in the first group
`pct.1`: The percentage of cells where the gene is detected in the first group
`pct.2`: The percentage of cells where the gene is detected in the second group
`p_val_adj`: Adjusted p-value, based on bonferroni correction using all genes in the dataset

### Data Viz

Seurat also comes with several functions for visualizing your data. You can use `ggplot` like syntax to tweak your figures as needed.

We can make a `FeaturePlot`, which uses the `data` slot by default. Not that if you split the plots by some `Ident`, the scales might be different across the different plots. Using the `theme(legend.position = "right")` to force Seurat to include a legend.

In [ ]:
#cluster_markers %>% dplyr::filter(pct.2 > .9) %>% dplyr::filter(avg_log2FC > 2)
FeaturePlot(all_data_integrated, features = 'IER2', reduction = 'tsne',  order = T, split.by = 'orig.ident') & 
   scale_colour_gradientn(colours = rev(brewer.pal(n = 11, name = "Spectral"))) & 
   theme(legend.position = "right")
#cluster_markers %>% dplyr::filter(pct.2 > .9) %>% dplyr::filter(avg_log2FC > 2)
FeaturePlot(all_data_integrated, features = 'IER2', reduction = 'tsne',  order = T, split.by = 'orig.ident') & 
   scale_colour_gradientn(limits = c(0,120), colours = rev(brewer.pal(n = 11, name = "Spectral"))) & 
   theme(legend.position = "right")

`RidgePlot` uses `counts` slot by default:

In [ ]:
RidgePlot(all_data_integrated, features = c('GAPDH','TPT1','GNAS','HLA-B', 'IER2'), ncol = 2) + theme(legend.position="none")


`DotPlot` uses the `data` slot, which is averaged and passed to scale. The size of each dot indicates that percentage of cells epxressing the feature, the color indicates the expression level.

In [ ]:
DotPlot(all_data_integrated, features = c('GAPDH','TPT1','GNAS','HLA-B','IER2'), assay = 'RNA')

### Cell type annotation